In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import seqtrainer.dataset_builder as dataset_builder
import seqtrainer.preprocessing as preprocessing
import os
import pandas as pd
import numpy as np

In [ ]:
current_dir = os.path.abspath('')
data_path = os.path.join(current_dir, '..', 'seqtrainer-data')
dataset_path = os.path.join(data_path, "sbol_data")

In [6]:
paths = [os.path.join(dataset_path, file_name) for file_name in os.listdir(dataset_path)][0:100]
paths


['c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_0.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_1.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_10.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_100.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_1000.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_10000.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_10001.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtrainer-dat\\sbol_data\\sample_design_10002.xml',
 'c:\\Users\\Sai\\Documents\\GitHub\\SBOLtrainer\\notebooks\\..\\seqtr

In [7]:
desired_y_uri = "http://www.ontology-of-units-of-measure.org/resource/om-2/hasNumericalValue"
df = dataset_builder.build_dataset(paths, desired_y_uri) 
df

,sequence,target
0,GCAAATTTTGCACAAAAAATAGGCTTTAGTGATTTGTTTTTGTTCA...,52.010547
1,GTATCTGCCTCCGATTCTCTGCAGAAGCAGAAAGACATTGGATCGA...,0.565387
2,CTTCCAGGCGGGTGGGGTCAATGTCCATCAGGGCAATATGCGCCGT...,0.690511
3,ATCAAAAATGAAGCCGATAACGGCCTGCGCAACACGCGTGGCACCA...,0.696464
4,TTCTTCATCGGGCAGATCTTCAAACGGTACCAGTCGGCGGACGTTT...,0.561674
...,...,...
95,CAGTTATTAATTTACCCCACGTCATCTGTCATAACACAGGTAACTT...,0.663684
96,GGTCCAATGTTTGTTGATCTGGATGATGCACGTAATGGTCCAGCCG...,0.626161
97,CTGCCGCAAAACATCACCTTAACCGAAGTGTAATTCTCTCTTGCCG...,0.911449
98,GCGACTCGGGTAAAAATAGTACACCTCGGTAGTGTCAAACATCATT...,0.675017


In [8]:
ohe_dna = preprocessing.process_seqs(df, 150, 'sequence')

In [9]:
y = df['target']
y  = np.log1p(y)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(ohe_dna, y, test_size=0.2, random_state=42)

In [11]:
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)   
mae = mean_absolute_error(y_test, y_pred)
print(f"Random Forest Regressor - MSE: {mse}, R2: {r2}, MAE: {mae}")

Random Forest Regressor - MSE: 0.6337398728096785, R2: -0.0494052482438303, MAE: 0.34963730962294964


In [12]:
gc = preprocessing.calc_gc(df, "sequence")
kmer = preprocessing.generate_kmer_counts(df, "sequence", 3)
X = pd.concat([gc, kmer], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)   
mae = mean_absolute_error(y_test, y_pred)
print(f"Linear Regression - MSE: {mse}, R2: {r2}, MAE: {mae}")